In [45]:
from py2neo import Graph
import pandas as pd

pd.set_option('display.max_colwidth', -1)

In [46]:
graph = Graph("bolt://localhost", auth=("neo4j", "neo"))

For this section we need to install the [ml-models](https://github.com/neo4j-graph-analytics/ml-models) procedures library. You can find installation instructions on the [releases pages](https://github.com/neo4j-graph-analytics/ml-models/releases/tag/1.0.1). 

Once you've done that, run the following code to check that the library is installed:

In [47]:
query = """
CALL dbms.procedures() 
YIELD name WHERE name STARTS WITH 'regression' 
RETURN *
"""

graph.run(query).to_data_frame()

,name
0,regression.linear.add
1,regression.linear.addM
2,regression.linear.clear
3,regression.linear.copy
4,regression.linear.create
5,regression.linear.delete
6,regression.linear.info
7,regression.linear.load
8,regression.linear.remove
9,regression.linear.removeM


Let's split training and test data:

In [48]:
split_data_train_query = """
MATCH (list:Listing)-[:IN_NEIGHBORHOOD]->(:Neighborhood) 
WHERE exists(list.bedrooms) AND exists(list.bathrooms)
AND exists(list.price) 
AND (:Review)-[:REVIEWS]->(list) 
WITH regression.linear.split(collect(id(list)), 0.75) AS trainingIDs
MATCH (list:Listing) WHERE id(list) in trainingIDs 
SET list:Train
"""

split_data_test_query = """
MATCH (list:Listing)-[n:IN_NEIGHBORHOOD]->(:Neighborhood)
WHERE exists(list.bedrooms) AND exists(list.bathrooms)
AND exists(list.price) 
AND (:Review)-[:REVIEWS]->(list) 
AND NOT list:Train 
SET list:Test
"""

graph.run(split_data_train_query).summary().counters
graph.run(split_data_test_query).summary().counters

{}

In [50]:
correlation_query = """
MATCH (list) 
WHERE list:Test OR list:Train
WITH collect(size((list)<-[:REVIEWS]-()) * 1.0) AS reviews,
     collect(list.bedrooms + list.bathrooms) as rooms
RETURN regression.linear.correlation(reviews, rooms)
"""

graph.run(correlation_query).to_data_frame()

,"regression.linear.correlation(reviews, rooms)"
0,0.026142


In [51]:
model_name = "rental-prices-4"

In [52]:
init_query = """
CALL regression.linear.create($modelName, 'Multiple', true, 2)
"""

graph.run(init_query, {"modelName": model_name}).summary().counters

{}

In [53]:
add_training_data_query = """
MATCH (list:Train)
WHERE NOT list:Seen 
CALL regression.linear.add($modelName, 
  [list.bedrooms + list.bathrooms, size((list)<-[:REVIEWS]-()) * 1.0], 
  list.price
) 
SET list:Seen 
RETURN count(list)
"""

graph.run(add_training_data_query, {"modelName": model_name}).summary().counters

{'labels_added': 36790}

In [54]:
train_model_query = """
CALL regression.linear.train($modelName)
"""

graph.run(train_model_query, {"modelName": model_name})

In [55]:
add_test_data_query = """
MATCH (list:Test) 
WHERE NOT list:Seen
CALL regression.linear.add('rental-prices-3', 
  [list.bedrooms + list.bathrooms, size((list)<-[:REVIEWS]-()) * 1.0],  
  list.price, 
  'test'
) 
SET list:Seen 
RETURN count(list)
"""

In [56]:
test_model_query = """
CALL regression.linear.test($modelName)
"""

graph.run(test_model_query, {"modelName": model_name}).to_data_frame()

,framework,hasConstant,model,nTest,nTrain,numVars,state,testInfo,trainInfo
0,Multiple,True,rental-prices-4,0,36790,2,ready,"{'adjRSquared': nan, 'RSquared': nan, 'SSE': 0.0, 'SST': 0.0, 'MSE': -0.0}","{'RSquared': 0.22357460363988557, 'SSR': 88897254.99853414, 'SSE': 308720602.9837708, 'SST': 397617857.98230493, 'adjRSquared': 0.22353239169564654, 'parameters std error': [1.2824171688063868, 0.5109515890197222, 0.011533589063208016], 'parameters': [16.223217245130986, 52.5881409596996, -0.034282521063706], 'MSE': 8392.111424790572}"


In [57]:
info_query = """
CALL regression.linear.info($modelName) 
"""

graph.run(info_query, {"modelName": model_name}).to_data_frame()

,framework,hasConstant,model,nTest,nTrain,numVars,state,testInfo,trainInfo
0,Multiple,True,rental-prices-4,0,36790,2,ready,"{'adjRSquared': nan, 'RSquared': nan, 'SSE': 0.0, 'SST': 0.0, 'MSE': -0.0}","{'RSquared': 0.22357460363988557, 'SSR': 88897254.99853414, 'SSE': 308720602.9837708, 'SST': 397617857.98230493, 'adjRSquared': 0.22353239169564654, 'parameters std error': [1.2824171688063868, 0.5109515890197222, 0.011533589063208016], 'parameters': [16.223217245130986, 52.5881409596996, -0.034282521063706], 'MSE': 8392.111424790572}"
